In [6]:
%matplotlib inline 
import numpy as np

import arrayfire as af
from scipy import ndimage
import matplotlib.pyplot as plt
from LearnAF import *
import tqdm
import pandas as pd

af.set_backend('opencl')

#DF = pd.read_csv('/home/narsi/Downloads/train.csv')
f = np.load('/home/narsi/Downloads/mnist.npz')
Data = f['x_train']
y_train = f['y_train']

Data = Data.reshape(60000, 784)
Data = Data.astype('float32')/255.0
classes = np.asarray(to_categorical(y_train), dtype = np.float32)

# initialize weights randomly with mean 0
syn0 = np.array(2*np.random.random((Data.shape[1],64)) - 1, dtype = np.float32)
W1 = Variable(af.np_to_af_array(syn0),name='W1')
b1 = Variable(af.constant(0,1),name='b1')

syn0 = np.array(2*np.random.random((64,16)) - 1, dtype = np.float32)
W2 = Variable(af.np_to_af_array(syn0),name='W2')
b2 = Variable(af.constant(0,1),name='b2')

syn0 = np.array(2*np.random.random((16,10)) - 1, dtype = np.float32)
W3 = Variable(af.np_to_af_array(syn0),name='W3')
b3 = Variable(af.constant(0,1),name='b3')

w = [W1,b1,W2,b2,W3,b3]

# Model

In [7]:
def learner(X,W):
    # 784 -> 64
    X1 = sigmoid(add(matmul(X,W[0]),W[1]))
    # 64 -> 16
    X2 = sigmoid(add(matmul(X1,W[2]),W[3]))
    # 16 -> 10
    YP = sigmoid(add(matmul(X2,W[4]),W[5]))
    return YP

# Place Holders

In [8]:
Xin = Constant(af.np_to_af_array(np.zeros((32,Data.shape[1]),dtype = np.float32)))
Y = Constant(af.np_to_af_array(np.zeros((32,10),dtype = np.float32)))

In [9]:
YP = learner(Xin,w)
e = mse(Y,YP)
acc = accuracy(Y,YP)
sgd = SGD(lr = 0.01,momentum=0.90)

In [10]:
for i in range(10):
    epoch_acc = []
    epoch_loss = []
    total_batchs = int(Data.shape[0]/32)
    for j in tqdm.tqdm(range(total_batchs)):
        X_np = Data[j*32:(j+1)*32,:]
        Xin = Constant(af.np_to_af_array(X_np))
        Y_np = classes[j*32:(j+1)*32,:]
        Y = Constant(af.np_to_af_array(Y_np))
        
        YP = learner(Xin,w)
        e = mse(Y,YP)
        acc = accuracy(Y,YP)
        (l,w) = sgd.update(e, w, i)
        epoch_acc.append(acc)
        epoch_loss.append(l)
        
    print('Accuracy :'+str(np.mean(epoch_acc)))
    print('Loss :'+str(np.mean(epoch_loss)))

  2%|▏         | 35/1875 [00:00<00:22, 83.05it/s]


RuntimeError: In function void opencl::evalNodes(std::vector<opencl::Param>&, std::vector<opencl::JIT::Node*>)
In file src/backend/opencl/jit.cpp:267
OpenCL Error (-42): Invalid Binary when calling clBuildProgram


In [4]:
from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD

model = Sequential()
model.add(Dense(64, activation='sigmoid', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(16, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='sigmoid'))

model.summary()

model.compile(loss='mse',
              optimizer=SGD(lr=0.01, momentum=0.9, decay=0.0, nesterov=False),
              metrics=['accuracy'])

history = model.fit(Data, classes,
                    batch_size=32,
                    epochs=10,
                    verbose=2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 64)                50240     
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 16)                1040      
_________________________________________________________________
dropout_6 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                170       
Total params: 51,450
Trainable params: 51,450
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
4s - loss: 0.1009 - acc: 0.1097
Epoch 2/10
4s - loss: 0.0913 - acc: 0.1185
Epoch 3/10
4s - loss: 0.0909 - acc: 0.1311
Epoch 4/1

KeyboardInterrupt: 

In [9]:

# input dataset
X = np.array([  [0,0,1],
                [0,1,1],
                [1,0,1],
                [1,1,1] ], dtype = np.float32)
X = Constant(af.np_to_af_array(X))
# output dataset            
y = np.array([[0,0,1,1]], dtype = np.float32).T
y = Constant(af.np_to_af_array(y)) 
# seed random numbers to make calculation
# deterministic (just a good practice)
np.random.seed(1)

# initialize weights randomly with mean 0
syn0 = np.array(2*np.random.random((3,10)) - 1, dtype = np.float32)
W1 = Variable(af.np_to_af_array(syn0),name='W1')
b1 = Variable(af.constant(0,1),name='b1')

syn0 = np.array(2*np.random.random((10,1)) - 1, dtype = np.float32)
W2 = Variable(af.np_to_af_array(syn0),name='W2')
b2 = Variable(af.constant(0,1),name='b2')

w = [W1,b1,W2,b2]

X1 = sigmoid(add(matmul(X,W1),b1))
yp = sigmoid(add(matmul(X1,W2),b2))
e = mse(y,yp)
acc = accuracy(y,yp)
sgd = SGD(lr = 0.001,momentum=0.99)